In [ ]:
import torch
from PIL import Image
from torchvision.transforms import v2
from diffusers.models import AutoencoderKL

# vae = AutoencoderKL.from_pretrained("ostris/vae-kl-f8-d16", torch_dtype=torch.float32)
vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", subfolder="vae", torch_dtype=torch.float32)

In [ ]:
img = Image.open('data/tet.jpg')

convert_tensor = v2.ToTensor()

tensor_img = convert_tensor(img)

print(f"Image shape: {tensor_img.shape}")

# add batch dim
tensor_img = tensor_img.unsqueeze(0).to('cuda')
print(tensor_img.min())
print(tensor_img.max())

vae.to("cuda")
vae.eval()
for param in vae.parameters():
    param.requires_grad = False
with torch.no_grad():
    sample = vae._encode(tensor_img)
    mean, logvar = torch.chunk(sample, 2, dim=1)
    eps = torch.randn_like(mean)
    z = mean #+ eps * torch.exp(0.5 * logvar)
    decoded_img = vae.decode(z)

print(decoded_img[0].shape)

import matplotlib.pyplot as plt
img_dec = decoded_img[0].reshape(3, 1936, 1944)
img_dec = img_dec.permute(1, 2, 0).cpu().numpy()
print(vae.config.scaling_factor)
plt.imshow(img_dec)